<a href="https://colab.research.google.com/github/mohamedsgap/practice-with-pytorch/blob/master/final_lab_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5770a000 @  0x7f4e7b5992a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [2]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.4.0:
  Successfully uninstalled Pillow-5.4.0
    100% |████████████████████████████████| 2.0MB 15.1MB/s 
4.0.0


In [0]:
# we will download the required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
#!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
# we will download the required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json

In [0]:
# Imports here

import numpy as np
import pandas as pd
import torch
import json
import torchvision
from PIL import Image
from collections import OrderedDict
import matplotlib.pyplot as plt

In [0]:
data_dir = '/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# TODO: Define your transforms for the training and validation sets
data_transforms =  {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.ColorJitter(hue=.06, saturation=.05),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ]),
    'valid': torchvision.transforms.Compose([        
        torchvision.transforms.Resize(224),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
}

# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train': torchvision.datasets.ImageFolder(
        'flower_data/train',
        transform=data_transforms['train']
    ),
    'valid': torchvision.datasets.ImageFolder(
        'flower_data/valid',
        transform=data_transforms['valid']
    )
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders =  {
    'train': torch.utils.data.DataLoader(
        image_datasets['train'],
        batch_size=256,
        shuffle=True,
        num_workers=16
    ),
    'valid': torch.utils.data.DataLoader(
        image_datasets['valid'],
        batch_size=256,
        shuffle=True,
        num_workers=16
    )
}


In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [0]:
# TODO: Build and train your network

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
def create_model(device=torch.device('cpu')):
    model = torchvision.models.resnet152(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
    
    num_features = model.fc.in_features
    
    fc_layers = torch.nn.Sequential(OrderedDict([
        ('fc1', torch.nn.Linear(num_features, 1024)),
        ('relu1', torch.nn.ReLU()),
        ('dropout1', torch.nn.Dropout(p=0.4)),
        ('fc2', torch.nn.Linear(1024, 512)),
        ('relu2', torch.nn.ReLU()),
        ('dropout2', torch.nn.Dropout(p=0.4)),
        ('fc3', torch.nn.Linear(512, 256)),
        ('relu3', torch.nn.ReLU()),
        ('dropout3', torch.nn.Dropout(p=0.4)),
        ('fc4', torch.nn.Linear(256, 102)),
        ('output', torch.nn.LogSoftmax(dim=1))
    ]))
    model.fc = fc_layers
    
    model.to(device)
    
    return model

In [11]:
model = create_model(device)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:06<00:00, 35650280.02it/s]


In [0]:
def train_model(densenet, criterion, optimizer, num_epochs):
    perfs = []
    
    best_loss = +np.inf
    dataset_sizes = {
        'train': len(image_datasets['train']),
        'valid': len(image_datasets['valid'])
    }
    
    for epoch in range(num_epochs):
        train_epoch_loss = 0
        train_epoch_acc = 0
        valid_epoch_loss = 0
        valid_epoch_acc = 0
        for phase in ['train', 'valid']:
            running_loss = 0.0
            running_corrects = 0
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                if phase == 'train':
                    densenet.train(True)
                else:
                    densenet.train(False)
                
                inputs = torch.autograd.Variable(inputs.to(device=device))
                labels = torch.autograd.Variable(labels.to(device=device))
                
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data).item()
                
                del inputs, labels, outputs, preds
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
                train_epoch_acc = running_corrects / dataset_sizes[phase]
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                valid_epoch_acc = running_corrects / dataset_sizes[phase]
        
        print('Epoch [{}/{}] train loss: {:.6f} acc: {:.6f} valid loss: {:.6f} acc: {:.6f}'.format(
            epoch + 1, num_epochs, train_epoch_loss, train_epoch_acc, valid_epoch_loss, valid_epoch_acc))
        
        perfs.append([train_epoch_loss, train_epoch_acc, valid_epoch_loss, valid_epoch_acc])
        
        if valid_epoch_loss < best_loss:
            print("Model improved (from loss: {:.6f} to {:.6f}), saving ".format(best_loss, valid_epoch_loss))
            best_loss = valid_epoch_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'class_to_idx': image_datasets['train'].class_to_idx
            }, 'checkpoint.pth.tar')
    
    print("Training completed - Best loss: {:.6f}".format(best_loss))
    perfs_df = pd.DataFrame(perfs, columns=['train_epoch_loss',
                                            'train_epoch_acc',
                                            'valid_epoch_loss',
                                            'valid_epoch_acc'])
    return perfs_df
               

In [0]:
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adadelta(model.fc.parameters(), lr=1.1, rho=0.8, eps=1e-07, weight_decay=0)

In [26]:
num_epochs = 200
perfs_df = train_model(model, criterion, optimizer, num_epochs)

AttributeError: ignored

In [25]:
%matplotlib inline

plt.figure(figsize=(20, 20))
plt.plot(np.arange(0, num_epochs), perfs_df['train_epoch_loss'], label='Train loss')
plt.plot(np.arange(0, num_epochs), perfs_df['train_epoch_acc'], label='Train accuracy')
plt.plot(np.arange(0, num_epochs), perfs_df['valid_epoch_loss'], label='Test loss')
plt.plot(np.arange(0, num_epochs), perfs_df['valid_epoch_acc'], label=' Test accuracy')
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

NameError: ignored

In [1]:
# TODO: Save the checkpoint 
""""
model.class_to_idx = train_data.class_to_idx
model.cpu
torch.save({'structure' :'densenet121',
            'hidden_layer1':120,
            'state_dict':model.state_dict(),
            'class_to_idx':model.class_to_idx},
            'checkpoint.pth')
            
            """

'"\nmodel.class_to_idx = train_data.class_to_idx\nmodel.cpu\ntorch.save({\'structure\' :\'densenet121\',\n            \'hidden_layer1\':120,\n            \'state_dict\':model.state_dict(),\n            \'class_to_idx\':model.class_to_idx},\n            \'checkpoint.pth\')\n            \n            '

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_checkpoint():
    model = create_model()
    optimizer = torch.optim.Adadelta(model.fc.parameters(), lr=1.1, rho=0.8, eps=1e-07, weight_decay=0)
    checkpoint = torch.load('checkpoint.pth.tar')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimizer

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    
    preprocess = torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    img = Image.open(image)
    img = preprocess(img)
    
    return img

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [0]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    
     
    model.eval()
    model.cpu()
    
    image = process_image(image_path)
    image = image.unsqueeze(0)
    image = torch.autograd.Variable(image)
    
    with torch.no_grad():
        output = model.forward(image)
        probs = torch.exp(output)
        top_probs, top_labels = probs.topk(topk)
    
    return torch.Tensor.numpy(top_probs[0]).tolist(), torch.Tensor.numpy(top_labels[0]).tolist()


In [0]:
# TODO: Display an image along with the top 5 classes

%matplotlib inline
img_path = 'flower_data/valid/102/image_08014.jpg'

model, optimizer = load_checkpoint()
top_probs, top_labels = predict(img_path, model, topk=5)
idx_to_class = {val: key for key, val in model.class_to_idx.items()}

plt.barh([cat_to_name[idx_to_class[lab]] for lab in top_labels],
        top_probs,
        align='center', color='blue')
plt.gca().invert_yaxis()
plt.show()

imshow(process_image(img_path))